In [1]:
import pandas as pd
import os
import xml.etree.cElementTree as ET
import re
import numpy as np
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [2]:
files = os.listdir("../coropus_for_korzun/")
files.sort()
Time = '{http://www.abbyy.com/ns/Time#}'
rdfs = '{http://www.w3.org/2000/01/rdf-schema#}'
Aux = '{http://www.abbyy.com/ns/Aux#}'

In [8]:
DATA = []
TOKENS = []
INDEXES = []
LABELS = []
prev_sent_num = 0

for fl in files:
    XML_FILE = os.path.join("../coropus_for_korzun", fl)

    tree = ET.ElementTree(file = XML_FILE)
    root = tree.getroot()
    dates = set()
    for child in root:
        if child.tag == Time + 'PointOfTime' or child.tag == Time + 'TimeInterval':
            children = child.getchildren()     
#             label = child.find(rdfs+'label')           
#             if label != None:                   
#                 dates.add(label.text) 
# #             else:
#                 try:
#                     dates.add(children[0].text)
#                 except Exception as e:  
#                     print(e)
            flag = False;       
            for part in children:            
                if part.tag == Time + 'month' or part.tag == Time + 'day' or part.tag == Time + 'year':
                    flag = True
                    break
            if flag == True:
                label = child.find(rdfs+'label')           
                if label != None:                   
                    dates.add(label.text) 
                else:
                    dates.add(children[0].text)
    text = root.find(Aux+'TextAnnotations').find(Aux+'document_text').text
#     print(dates)

    sentences = sent_tokenize(text.replace('\u2028', ' '))
    words_by_sentence = []
    for sentence in sentences:
        words_by_sentence.append(word_tokenize(sentence))

    splitted_dates = []
    for date in dates:
        splitted_dates.append(word_tokenize(date))     
#     print(words_by_sentence)


    labels_by_sentence = []
    indexes_by_sentence = []
    for w in range(len(words_by_sentence)):
        words = words_by_sentence[w]
        indexes = len(words)*[prev_sent_num+w] 
        labels = len(words)*['NONE']
        for i in range(len(words)):
            for j in range(len(splitted_dates)):
                if words[i] == splitted_dates[j][0] and labels[i] =='NONE':
                    date_length = len(splitted_dates[j])
                    flag = True
                    for k in range(date_length):
                        try:
                            if words[i+k] != splitted_dates[j][k]:
                                flag = False
                        except Exception as e:
                            flag = False
                    if flag == True:
                        if date_length == 1:
                            labels[i] = 'D_OR'
                        else:
                            labels[i] = 'D_ST'
                            for k in range(len(splitted_dates[j]) - 1 ):
                                labels[i+1+k] = 'D_PT'
                            labels[i+len(splitted_dates[j]) - 1] = 'D_EN'
        indexes_by_sentence.append(indexes)
        labels_by_sentence.append(labels)
    prev_sent_num+=len(words_by_sentence)

    # print(indexes_by_sentence)
    # print(labels_by_sentence)

    
    for i in range(len(indexes_by_sentence)):
        for j in range(len(indexes_by_sentence[i])):
            DATA.append([words_by_sentence[i][j], indexes_by_sentence[i][j]])
            TOKENS.append(words_by_sentence[i][j])
            INDEXES.append(indexes_by_sentence[i][j])
            LABELS.append(labels_by_sentence[i][j])

In [9]:
df = pd.DataFrame(data= DATA, index = LABELS)                        
df.to_csv('dates.csv') 